<a href="https://colab.research.google.com/github/VigneshBaskar/forfun/blob/master/Copy_of_ECCV2022_Implicitron_config_PUBLIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Implicitron Config system

Goals of the config system:
1. Config schema and defaults are defined in the code of the classes; the overridden keys and types of the values are validated before running the code.
2. It should be easy to override a parameter deep into the class hierarchy, no need to propagate it through the layers.
3. The config system is not intrusive: the classes can be used to instantiate objects in a normal way (with some exceptions).



## 0. Install and import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:


In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath visdom==0.1.8.9 omegaconf
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
# prepare the data
!wget https://dl.fbaipublicfiles.com/pytorch3d/data/implicitron_tutorial/nerf-synthetic-chair.tar.gz
!tar -xzf nerf-synthetic-chair.tar.gz

In [ ]:
from typing import Dict, Optional, Tuple
import numpy as np
import matplotlib.cm
import imageio
import torch
import torch.nn.functional as F
import IPython
from omegaconf import OmegaConf
from PIL import Image
from pytorch3d.implicitron.tools.config import Configurable, expand_args_fields, get_default_args, registry, run_auto_creation
from pytorch3d.implicitron.dataset.data_source import (
    DataSourceBase,
    ImplicitronDataSource,
)
from pytorch3d.implicitron.models.generic_model import ImplicitronModelBase, GenericModel
from pytorch3d.implicitron.models.implicit_function.base import ImplicitFunctionBase
from pytorch3d.implicitron.models.renderer.base import ImplicitronRayBundle
from pytorch3d.renderer import CamerasBase

In [ ]:
class ToyExperiment(Configurable):
  height: int = 5
  width: int = 7

  def run(self):
    print(self.height * self.width)

In [ ]:
defaults = get_default_args(ToyExperiment)
print(type(defaults), defaults)

Now you can instantiate the object from a dict config where data types and keys have been verified to match `ToyExperiment`.

In [ ]:
ToyExperiment(**defaults).run()

We often override parameters from CLI or a yaml file after parsing them with `OmegaConf.from_dotlist` or `OmegaConf.create`, respectively. The former can be done e.g. like:

```
python experiment.py height=3 
```

If you use Hydra, it will do that for you.

In [ ]:
override = OmegaConf.from_dotlist(["height=3"])
updated = OmegaConf.merge(defaults, override)
print(updated)
ToyExperiment(**updated).run()

In [ ]:
# type checking; EITHER LINE IS SUPPOSED TO RAISE AN ERROR

OmegaConf.merge(defaults, OmegaConf.from_dotlist(["not_a_param=5"]))
OmegaConf.merge(defaults, OmegaConf.from_dotlist(["height=NotInteger"]))

## 1. Default Implicitron Experiment config

In pracrice, the object structure (and thus configs) form a hierarchy. Let’s see how it is used in Implicitron. The top-level config may be defined like this:


In [ ]:
class Experiment(Configurable):
    data_source: DataSourceBase
    data_source_class_type: str = "ImplicitronDataSource"
    model_factory: ImplicitronModelBase
    model_factory_class_type: str = "GenericModel"

    def __post_init__(self):
        run_auto_creation(self)

    def run(self):
        # your training loop caling `model.forward()`
        pass

Let’s check what those classes are:

```python
class DataSourceBase(ReplaceableBase):
  ...

@registry.register
class ImplicitronDataSource(DataSourceBase):
    dataset_map_provider: DatasetMapProviderBase
    dataset_map_provider_class_type: str = ""
    data_loader_map_provider: DataLoaderMapProviderBase
    data_loader_map_provider_class_type: str = "SequenceDataLoaderMapProvider"
  ...
```

In [ ]:
defaults = get_default_args(Experiment)

In [ ]:
print(OmegaConf.to_yaml(defaults)[:600] + "\n...")

Some things to note here:

1. The default config picked up all available implementations of ReplaceableBase classes, e.g. for `dataset_map_provider`, it found in the registry `BlenderDatasetMapProvider` and `JsonIndexDatasetMapProvider`.
2. There are the fields with `???` that have to be overriden (e.g. from CLI or yaml file) before instantiating. Let’s do that:

## 2. Overridding defaults and instantiating the hierarchy

In [ ]:
override_dotlist = OmegaConf.from_dotlist([
    "data_source_ImplicitronDataSource_args.dataset_map_provider_class_type=BlenderDatasetMapProvider",
])

# we can also override them in yaml, which looks more concise
override_yaml = OmegaConf.create("""
data_source_ImplicitronDataSource_args:
    dataset_map_provider_BlenderDatasetMapProvider_args:
        base_dir: ./chair
        object_name: chair
""")
updated = OmegaConf.merge(defaults, override_yaml, override_dotlist)

In [ ]:
print(OmegaConf.to_yaml(updated)[:500] + "\n...")

In [ ]:
# this creates the objects all the way down!
instance = Experiment(**updated)

In [ ]:
datasets, dataloaders = instance.data_source.get_datasets_and_dataloaders()


In [ ]:
print(datasets["train"][32].camera.R)

In [ ]:
Image.fromarray((datasets["train"][32].image_rgb.permute(1, 2, 0).data * 255.).byte().cpu().numpy())

## 3. `implicitron_trainer` project

We provide a template Experiment definition and the set of configs to run the common methods (NeRF, IDR, SRN, etc.) in `pytorch3d/projects/implicitron_trainer/experiment.py`. You can reuse it or take as a starting point. For example, to train a NeRF on a CO3D scene, you can run

```sh
dataset_args=data_source_ImplicitronDataSource_args.dataset_map_provider_JsonIndexDatasetMapProvider_args
pytorch3d_implicitron_runner --config-path ./configs/ --config-name repro_singleseq_nerf \
    $dataset_args.dataset_root=<DATASET_ROOT> $dataset_args.category='skateboard' \
    $dataset_args.test_restrict_sequence_id=0 test_when_finished=True exp_dir=<CHECKPOINT_DIR>
```

NOTE: this section is NOT self-contained and is not wupposed to be executed.

#### Use case: implement a new implicit function.

In [ ]:
@registry.register
class MyImplicitFunction(ImplicitFunctionBase, torch.nn.Module):
    # parameters...

    def __post_init__(self) -> None:
        super().__init__()  # this is necessary if we derive from nn.Module
        run_auto_creation(self)

    def forward(
        self,
        ray_bundle: ImplicitronRayBundle,
        fun_viewpool=None,
        camera: Optional[CamerasBase] = None,
        global_code=None,
        **kwargs,
    ) -> Tuple[torch.Tensor, torch.Tensor, Dict]:
        # evaluate opacity and colour in the points defined by ray_bundle
        opacity = torch.empty()
        color = torch.empty()
        return opacity, color, {}

Import this code from `experiment.py`. You can now enable this implicit function by setting the key
`model_factory_ImplicitronModelFactory_args.model_GenericModel_args.implicit_function_class_type=MyImplicitFunction`

#### Use case: define a new dataset.

Imageine you have made a several photos and estimated cameras with COLMAP.
They are stored in files `image_000.jpg`, `image_001.jpg`, ..., and the corresponding estimated viewpoints are stored in the COLMAP-produced binary file in OpenCV format.

In [ ]:
import torchvision
from pytorch3d.utils import cameras_from_opencv_projection
from pytorch3d.implicitron.dataset.dataset_map_provider import DatasetMap, DatasetMapProviderBase, SingleSceneDataset, DATASET_TYPE_KNOWN
from .colmap_scripts import read_write_model

In [ ]:
def read_cameras(colmap_scene_dir):
    # read_model is provided by COLMAP:
    # https://github.com/colmap/colmap/blob/dev/scripts/python/read_write_model.py
    calibs, images, points3D = read_write_model.read_model(colmap_scene_dir, ".bin")
    cameras = {}
    for image_meta in images.values():
        intrinsic = calibs[image_meta.camera_id]
        assert intrinsic.model == "OPENCV"
        # qvec2rotmat is another function from read_write_model.py
        R = read_write_model.qvec2rotmat(image_meta.qvec)[:3, :3]
        t = np.array(image_meta.tvec)

        fx, fy, p0x, p0y = intrinsic.params[:4]
        distortion_coeffs_np = intrinsic.params[4:]
        imsize = np.array([intrinsic.height, intrinsic.width])
        K = np.eye(3).astype(np.float32)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = p0x
        K[1, 2] = p0y

        camera_pt3d = cameras_from_opencv_projection(
            torch.from_numpy(R)[None].float(),
            torch.from_numpy(t)[None].float(),
            torch.from_numpy(K)[None].float(),
            torch.from_numpy(imsize)[None].long(),
        )
        cameras[image_meta.name] = camera_pt3d
    
    return cameras

In [ ]:
@registry.register
class MyDataset(DatasetMapProviderBase):
    base_dir: str
    colmap_scene_dir: str

    # load data here
    def __post_init__(self) -> None:
        run_auto_creation(self)

        image_files = sorted(fname for fname in os.listdir(self.base_dir) if fname.startswith("image"))
        self.images = [torchvision.io.read_image(fname) for fname in image_files]
        cameras_dict = read_cameras(self.colmap_scene_dir)
        self.cameras = [cameras_dict[image] for image in image_files]
        

    def get_dataset_map(self) -> DatasetMap:
        return DatasetMap(
            train=self._get_dataset(),  # for simplicity, we do not define splits
            val=self._get_dataset(),
            test=self._get_dataset(),
        )

    def _get_dataset(self):
        return SingleSceneDataset(
            object_name="my_object",
            images=self.images,
            poses=self.cameras,
            frame_types=[DATASET_TYPE_KNOWN] * len(self.images),
        )

Import this code from `experiment.py`. You can now enable this implicit function by setting the key

`data_source_ImplicitronDataSource_args.dataset_map_provider_class_type=MyDataset`

